In [ ]:
import numpy as np

# **RPPG 파일 전처리**

In [ ]:
# /content/drive/MyDrive/testCode/EMMA/Experiment/IITP_TEST/Senior_Data_Excel/231221/CTB_0922/arima_rppg_databpm.csv

import csv

filename = '/content/drive/MyDrive/testCode/EMMA/Experiment/IITP_TEST/Senior_Data_Excel/231221/SIH_0203/arima_rppg_databvp[ppg]2.csv'  # 1814초 30분:14초 /  실제실험시간 : 1:00 - 29:10
data = []

with open(filename, 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)

    # 각 행(row)에 대하여 반복
    for row in csv_reader:
      data.append(row)
      #print(row)  # 현재 행 출력

print("공백줄 제거 전", len(data))
rPPG_data = [i for i in data if len(i) != 0]
print("공백줄 제거 후", len(rPPG_data)) # 1814 * 15

In [ ]:
for i in rPPG_data:
  #print(len(i[0]), end=' : ')
  i[0] = i[0].replace('[', "")
  i[0] = i[0].replace(']', "")
  if i[0] == 'nan':
    i[0] = [0]

In [ ]:
for i in rPPG_data:
  if i[0] != [0]:
    i[0] = [float(i) for i in i[0].split()]

1814 second
1초 => frame 15장 X rppg 90개

ppg
1초 => 30개

In [ ]:
rPPG_data_second = [rPPG_data[i:i+15] for i in range(0, len(rPPG_data), 15)]
len(rPPG_data_second)

In [ ]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

'''
# 예제 중첩 리스트
nested_list = rPPG_data

# 리스트 평탄화 실행
flat_list = flatten_list(nested_list)

print(len(flat_list))
'''

'''
import plotly.graph_objects as go

flat_list = flat_list[:10000]

# 데이터 준비
x_values = [i for i in range(len(flat_list))]
y_values = flat_list

# 그래프 객체 생성
fig = go.Figure()

# 선 그래프 추가
fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='rPPG'))

# 그래프 제목 및 축 레이블 설정
fig.update_layout(title='Plotly 선 그래프 예제',
                  xaxis_title='X 축',
                  yaxis_title='Y 축')

# 그래프 보여주기
fig.show()
'''


In [ ]:
rPPG_data_second_15 = []
for i in range(1815):
  rPPG_data_second_15.append(flatten_list(rPPG_data_second[i]))

In [ ]:
len(rPPG_data_second_15)

In [ ]:
len(rPPG_data_second_15[150:1750])

# **PPG Label**

In [ ]:
# /content/drive/MyDrive/testCode/EMMA/Experiment/IITP_TEST/CTB_0922.txt
filename = '/content/drive/MyDrive/testCode/EMMA/Experiment/IITP_TEST/SIH_0203.txt'  # 텍스트 파일의 경로
label_data = []
switch_data = []
with open(filename, 'r', encoding='utf-8') as file:
    for line in file:
        #print(line.strip())  # strip() 함수로 줄 바꿈 문자 제거 후 출력
        label_data.append(line.strip().split('\t')[0])
        switch_data.append(line.strip().split('\t')[1])

In [ ]:
label_data_float = [float(i) for i in label_data]
label_switch_float = [float(i) for i in switch_data]
len(label_data), len(label_switch_float)

In [ ]:
48005 / 30 # 49064 - 1059 스위치 시작 끝

In [ ]:
import plotly.graph_objects as go

# 데이터 리스트
list1 = label_data_float
list2 = label_switch_float

# 첫 번째 리스트를 위한 추적 객체 생성
trace1 = go.Scatter(
    x=list(range(len(list1))),
    y=list1,
    mode='lines',
    name='PPG'
)

# 두 번째 리스트를 위한 추적 객체 생성
trace2 = go.Scatter(
    x=list(range(len(list2))),
    y=list2,
    mode='lines',
    name='SWITCH'
)

# 두 추적 객체를 하나의 그래프로 합치기
data = [trace1, trace2]

# 그래프 그리기
fig = go.Figure(data=data)
fig.show()


In [ ]:
label_data_float_second = []
for i in range(0, len(label_data_float[1059:49064]), 30):
  label_data_float_second.append(label_data_float[1059:49064][i:i+30])

len(label_data_float_second)

In [ ]:
np.shape(label_data_float_second[:1600]), np.shape(rPPG_data_second_15[150:1750]) # PPG, rPPG

In [ ]:
PPG_DATA = label_data_float_second[:1600]
RPPG_DATA = rPPG_data_second_15[150:1750]

# **PPG&rPPG -> Heart rate**

In [ ]:
!pip install heartpy

In [ ]:
import heartpy as hp
import matplotlib.pyplot as plt
import numpy as np

ppg_bpm_list = []
ppg_sdnn_list = []

for i in range(len(PPG_DATA)):
  data, timer = flatten_list(PPG_DATA[i:i+5]), []
  #print(np.shape(data))
  try:
    wd, m = hp.process(data, sample_rate = 30.0)
  except:
    continue

  for measure in m.keys():
    if measure == 'bpm':
      ppg_bpm_list.append(m[measure])
    if measure == 'sdnn':
      ppg_sdnn_list.append(m[measure])
    #print('%s: %f' %(measure, m[measure]))

# call plotter
# hp.plotter(wd, m)

print(len(ppg_bpm_list))
print(len(ppg_sdnn_list))

#plt.figure(figsize=(12,4))
#plt.plot(data)
#plt.show()

In [ ]:
import heartpy as hp
import matplotlib.pyplot as plt
import numpy as np

rppg_bpm_list = []
rppg_sdnn_list = []

for i in range(len(RPPG_DATA)):
  data, timer = flatten_list(RPPG_DATA[i:i+5]), []
  #print(np.shape(data))
  try:
    wd, m = hp.process(data, sample_rate = 30.0)
  except:
    continue
  #hp.plotter(wd, m)

  for measure in m.keys():
    if measure == 'bpm':
      rppg_bpm_list.append(m[measure])
    if measure == 'sdnn':
      rppg_sdnn_list.append(m[measure])
    #print('%s: %f' %(measure, m[measure]))

# call plotter
# hp.plotter(wd, m)

print(len(rppg_bpm_list))
print(len(rppg_sdnn_list))

#plt.figure(figsize=(12,4))
#plt.plot(data)
#plt.show()

In [ ]:
data, timer = flatten_list(RPPG_DATA[:1]), []
#print(np.shape(data))
wd, m = hp.process(data, sample_rate = 1350)

hp.plotter(wd, m)

# **시각화**

In [ ]:
import plotly.graph_objects as go
import numpy as np

data1 = ppg_bpm_list[:1593]
data2 = rppg_bpm_list

# Plotly를 사용하여 그래프 그리기
fig = go.Figure()

# 데이터 1 추가
fig.add_trace(go.Scatter(y=data1, mode='lines', name='ppg2hr'))

# 데이터 2 추가
fig.add_trace(go.Scatter(y=data2, mode='lines', name='rppg2hr'))

# 그래프 제목과 축 이름 설정
fig.update_layout(title='PPG/RPPG TO BPM',
                  xaxis_title='Data Point Index(Second)',
                  yaxis_title='Value')

# 그래프 보기
fig.show()